# Музыка больших городов

## Введение

**Цель исследования**: сравнение музыкальных предпочтений музыкального сервиса для жителей Москвы и Санкт-Петербурга в разное время дня и разные дни недели.


**Входные данные** 

Информация о пользователях сервисом (*/datasets/music_project.csv*):

* userID — идентификатор пользователя;
* Track — название трека;  
* artist — имя исполнителя;
* genre — название жанра;
* City — город, в котором происходило прослушивание;
* time — время, в которое пользователь слушал трек;
* Day — день недели.

Вопросы для анализа:  
* как музыка, которая звучит по дороге на работу в понедельник утром, отличается от той, что играет в среду или в конце рабочей недели? 
* сравнить, что и в каком режиме слушают жители Москвы и Санкт-Петербурга.  

Рабочие гипотезы:
1. музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;
2. списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;
3. население двух городов предпочитает разные музыкальные жанры.

# 1. Получение и  обработка данных

Изучим данные, предоставленные сервисом для проекта:

In [1]:
import pandas as pd # <импорт библиотеки pandas>

Прочитаем файл music_project.csv и сохраним его в переменной df.

In [2]:
df = pd.read_csv('/datasets/music_project.csv')
df.head(10) # <получение первых 10 строк таблицы df>

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


In [3]:
df.info() # <получение общей информации о данных в таблице df>

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
  userID    65079 non-null object
Track       63848 non-null object
artist      57876 non-null object
genre       63881 non-null object
  City      65079 non-null object
time        65079 non-null object
Day         65079 non-null object
dtypes: object(7)
memory usage: 3.5+ MB


Каждая строка таблицы содержит информацию о композициях определённого жанра в определённом исполнении, которые пользователи слушали в одном из городов в определённое время и день недели. На текущем этапе уже можно определить две явные проблемы, которые требуется решать: пропуски и некачественные названия столбцов.  
Для проверки рабочих гипотез особенно ценны столбцы *time*, *day* и *City*. Данные из столбца *genre* позволят узнать самые популярные жанры.  

Исключим пропуски, переименуем столбцы, а также проверим данные на наличие дубликатов.

In [4]:
print(df.columns)

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')


В названиях столбцов есть пробелы, которые могут затруднять доступ к данным. Переименуем столбцы для удобства дальнейшей работы. Проверим результат.

In [5]:
df.set_axis(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time', 'weekday'], 
            axis = 'columns', inplace = True) # <переименование столбцов>
df.columns

Index(['user_id', 'track_name', 'artist_name', 'genre_name', 'city', 'time',
       'weekday'],
      dtype='object')

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [6]:
df.isnull().sum()

user_id           0
track_name     1231
artist_name    7203
genre_name     1198
city              0
time              0
weekday           0
dtype: int64

Пустые значения свидетельствуют, что для некоторых треков доступна не вся информация. Причины могут быть разные: вероятно, не назван конкретный исполнитель народной песни; также возможно, что проблемы с записью данных.  

Заменим пропущенные значения в столбцах с названием трека и исполнителя на строку 'unknown' и удалим в столбце с жанрами, после чего убедимся, что  таблица больше не содержит пропусков.

In [7]:
df['track_name'] = df['track_name'].fillna('unknown') # <замена пропущенных значений в столбце 'track_name' на строку 'unknown' специальным методом замены>
df['artist_name'] = df['artist_name'].fillna('unknown') # <замена пропущенных значений в столбце 'artist_name' на строку 'unknown' специальным методом замены>
df.dropna(subset = ['genre_name'], inplace = True) # <удаление пропущенных значений в столбце 'genre_name'>
df.isnull().sum() # <проверка: вычисление суммарного количества пропусков, выявленных в таблице df>

user_id        0
track_name     0
artist_name    0
genre_name     0
city           0
time           0
weekday        0
dtype: int64

Установим наличие полных дубликатов. Если найдутся - удаляем.

In [8]:
df.duplicated().sum()

3755

In [9]:
df = df.drop_duplicates().reset_index(drop = True)
df.duplicated().sum()

0

Дубликаты могли появиться вследствие сбоя в записи данных. Стоит уточнить у бизнеса, о возникновении причин появления такого «информационного мусора».  

Сохраним список уникальных значений столбца с жанрами в переменной `genres_list`.

Объявим функцию `find_genre()` для поиска неявных дубликатов в столбце с жанрами. Например, когда название одного и того же жанра написано разными словами.

In [10]:
genres_list = df['genre_name'].unique() 
# <сохранение в переменной genres_list списка уникальных значений, выявленных специальным методом в столбце 'genre_name'>
print(genres_list)

['rock' 'pop' 'folk' 'dance' 'rusrap' 'ruspop' 'world' 'electronic'
 'alternative' 'children' 'rnb' 'hip' 'jazz' 'postrock' 'latin'
 'classical' 'metal' 'reggae' 'tatar' 'blues' 'instrumental' 'rusrock'
 'dnb' 'türk' 'post' 'country' 'psychedelic' 'conjazz' 'indie'
 'posthardcore' 'local' 'avantgarde' 'punk' 'videogame' 'techno' 'house'
 'christmas' 'melodic' 'caucasian' 'reggaeton' 'soundtrack' 'singer' 'ska'
 'shanson' 'ambient' 'film' 'western' 'rap' 'beats' "hard'n'heavy"
 'progmetal' 'minimal' 'contemporary' 'new' 'soul' 'holiday' 'german'
 'tropical' 'fairytail' 'spiritual' 'urban' 'gospel' 'nujazz' 'folkmetal'
 'trance' 'miscellaneous' 'anime' 'hardcore' 'progressive' 'chanson'
 'numetal' 'vocal' 'estrada' 'russian' 'classicmetal' 'dubstep' 'club'
 'deep' 'southern' 'black' 'folkrock' 'fitness' 'french' 'disco'
 'religious' 'hiphop' 'drum' 'extrememetal' 'türkçe' 'experimental' 'easy'
 'metalcore' 'modern' 'argentinetango' 'old' 'breaks' 'eurofolk'
 'stonerrock' 'industrial' 'fu

In [11]:
def find_genre(genre): # <создание функции find_genre()>
    count = 0
    for row in genres_list:
        if row == genre:
            count += 1
    return count # по окончании работы цикла функция возвращает значение счётчика

Вызов функции *find_genre()* для поиска различных вариантов названия жанра хип-хоп в таблице.

Правильное название — *hiphop*. Найдем другие варианты:

* hip
* hop
* hip-hop

In [12]:
print(find_genre('hip'))
print(find_genre('hop'))
print(find_genre('hip-hop'))

1
0
0


Объявим функцию *find_hip_hop()*, которая заменяет  неправильное название этого жанра в столбце *'genre_name'* на *'hiphop'* и проверяет успешность выполнения замены.

Так исправляем все варианты написания, которые выявила проверка.

In [13]:
def find_hip_hop(genres, wrong): # <создание функции find_hip_hop()>
    genres['genre_name'] = genres['genre_name'].replace(wrong, 'hiphop')
    count =  genres['genre_name'][genres['genre_name'] == wrong].count()
    return count

In [14]:
find_hip_hop(df, 'hip')

0

Проверим общую информацию о данных и убеждаемся, что чистка выполнена успешно.

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60126 entries, 0 to 60125
Data columns (total 7 columns):
user_id        60126 non-null object
track_name     60126 non-null object
artist_name    60126 non-null object
genre_name     60126 non-null object
city           60126 non-null object
time           60126 non-null object
weekday        60126 non-null object
dtypes: object(7)
memory usage: 3.2+ MB


**Вывод**  
На этапе предобработки в данных обнаружились не только пропуски и проблемы с названиями столбцов, но и различного вида дубликаты. Их удаление позволит провести анализ точнее. Поскольку сведения о жанрах важно сохранить для анализа, не просто удаляем все пропущенные значения, но заполним пропущенные имена исполнителей и названия треков. Имена столбцов теперь корректны и удобны для дальнейшей работы.

## Анализ данных

### Слушают ли музыку в разных городах по-разному?

Была выдвинута гипотеза, что в Москве и Санкт-Петербурге пользователи слушают музыку по-разному. Проверяем это предположение по данным о трёх днях недели — понедельнике, среде и пятнице.

Для каждого города устанавливаем количество прослушанных  в эти дни композиций с известным жанром, и сравниваем результаты.  

Группируем данные по городу и вызовом метода *count()* подсчитываем композиции, для которых известен жанр.

In [16]:
df.groupby('city')['genre_name'].count() 
# <группировка данных таблицы df по столбцу 'city' и подсчёт количества значений столбца 'genre_name'>

city
Moscow              41892
Saint-Petersburg    18234
Name: genre_name, dtype: int64

В Москве прослушиваний больше, чем в Питере, однако это не значит, что Москва более активна. У сервиса в целом больше пользователей в Москве, поэтому величины сопоставимы.  

Сгруппируем данные по дню недели и подсчитаем прослушанные в понедельник, среду и пятницу композиции, для которых известен жанр.

In [17]:
df.groupby('weekday')['genre_name'].count() # <группировка данных по столбцу 'weekday' и подсчёт количества значений столбца 'genre_name'>


weekday
Friday       21482
Monday       20866
Wednesday    17778
Name: genre_name, dtype: int64

Понедельник и пятница — время для музыки; по средам пользователи немного больше вовлечены в работу.  
Создаём функцию *number_tracks()*, которая принимает как параметры таблицу, день недели и название города, а возвращает количество прослушанных композиций, для которых известен жанр. Проверяем количество прослушанных композиций для каждого города и понедельника, затем среды и пятницы.

In [18]:
def number_tracks(df, day, city): # <создание функции number_tracks()>
    track_list = df[(df['weekday'] == day) & (df['city'] == city)]
    track_list_count = track_list['genre_name'].count()
    return track_list_count
# функция возвращает значение track_list_count

In [19]:
msk_mo = number_tracks(df, 'Monday', 'Moscow') # <список композиций для Москвы в понедельник>
spb_mo = number_tracks(df, 'Monday', 'Saint-Petersburg') # <список композиций для Санкт-Петербурга в понедельник>
msk_we = number_tracks(df, 'Wednesday', 'Moscow') # <список композиций для Москвы в среду>
spb_we = number_tracks(df, 'Wednesday', 'Saint-Petersburg') # <список композиций для Санкт-Петербурга в среду>
msk_fr = number_tracks(df, 'Friday', 'Moscow')# <список композиций для Москвы в пятницу>
spb_fr = number_tracks(df, 'Friday', 'Saint-Petersburg') # <список композиций для Санкт-Петербурга в пятницу>

Сведём полученную информацию в одну таблицу, где ['city', 'monday', 'wednesday', 'friday'] названия столбцов.


In [20]:
data_fin = [['Moscow', msk_mo, msk_we, msk_fr], ['Saint-Petersburg', spb_mo, spb_we, spb_fr]] 
columns_fin = ['city', 'monday', 'wednesday', 'friday'] 
table = pd.DataFrame(data = data_fin, columns = columns_fin)
display(table)

,city,monday,wednesday,friday
0,Moscow,15347,10865,15680
1,Saint-Petersburg,5519,6913,5802


**Вывод:**  Результаты показывают, что относительно среды музыку в Петербурге и Москве слушают «зеркально»: в Москве пики приходятся на понедельник и пятницу, а в среду время прослушивания снижается. Тогда как в Санкт-Петербурге среда — день самого большого интереса к музыке, а в понедельник и пятницу он меньше, причём почти одинаково меньше.

### Разная ли музыка утром понедельника и вечером пятницы?

Ищем ответ на вопрос, какие жанры преобладают в разных городах в понедельник утром и в пятницу вечером. Есть предположение, что в понедельник утром пользователи слушают больше бодрящей музыки (например, жанра поп), а вечером пятницы — больше танцевальных (например, электронику).  

Получим таблицы данных по Москве *moscow_general* и по Санкт-Петербургу *spb_general*.

In [21]:
moscow_general = df[df['city'] == 'Moscow'] # получение таблицы moscow_general из тех строк таблицы df, 
# для которых значение в столбце 'city' равно 'Moscow'

In [22]:
spb_general = df[df['city'] == 'Saint-Petersburg'] # <получение таблицы spb_general>

Создаём функцию *genre_weekday()*, которая возвращает список жанров по запрошенному дню недели и времени суток с такого-то часа по такой-то.

In [23]:
def genre_weekday(df, day, time1, time2):
    genre_list = df[(df['weekday'] == day) & (df['time'] > time1) & (df['time'] < time2)] 
    genre_list_sorted = genre_list.groupby('genre_name')['track_name'].count().sort_values(ascending=False).head(10)
    return genre_list_sorted

Cравниваем полученные результаты по таблице для Москвы и Санкт-Петербурга в понедельник утром (с 7 до 11) и в пятницу вечером (с 17 до 23).

In [24]:
genre_weekday(moscow_general, 'Monday', '07:00:00', '11:00:00')

genre_name
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
classical      157
Name: track_name, dtype: int64

In [25]:
genre_weekday(spb_general, 'Monday', '07:00:00', '11:00:00')

genre_name
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: track_name, dtype: int64

In [26]:
genre_weekday(moscow_general, 'Friday', '17:00:00', '23:00:00')

genre_name
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: track_name, dtype: int64

In [27]:
genre_weekday(spb_general, 'Friday', '17:00:00', '23:00:00')# <вызов функции для вечера пятницы в Питере>

genre_name
pop            256
rock           216
electronic     216
dance          210
hiphop          97
alternative     63
jazz            61
classical       60
rusrap          59
world           54
Name: track_name, dtype: int64

Популярные жанры в понедельник утром в Питере и Москве оказались похожи: везде, как и предполагалось, популярен поп. Несмотря на это, концовка топ-10 для двух городов различается: в Питере в топ-10 входит джаз и русский рэп, а в Москве жанр *world*.

В конце недели ситуация не меняется. Поп-музыка всё так же на первом месте. Опять разница заметна только в концовке топ-10, где в Питере пятничным вечером тоже присутствует жанр *world*.

**Вывод**: Жанр поп безусловный лидер, а топ-5 в целом не различается в обеих столицах. При этом видно, что концовка списка более «живая»: для каждого города выделяются более характерные жанры, которые действительно меняют свои позиции в зависимости от дня недели и времени.

### Москва и Санкт-Петербург — две разные столицы с разными направлениями в музыке

Гипотеза: Санкт-Петербург богат своей рэп-культурой, поэтому это направление там слушают чаще, а Москва — город контрастов, но основная масса пользователей слушает поп-музыку.  

Сгруппируем таблицу *moscow_general* по жанру, сосчитаем численность композиций каждого жанра методом *count()*, отсортируем в порядке убывания и сохраним результат в таблице *moscow_genres*.

Просмотрим первые 10 строк этой новой таблицы.

In [28]:
moscow_genres = moscow_general.groupby('genre_name')['track_name'].count().sort_values(ascending=False)
display(moscow_genres.head(10))

genre_name
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161
Name: track_name, dtype: int64

In [29]:
spb_genres = spb_general.groupby('genre_name')['track_name'].count().sort_values(ascending=False) 
display(spb_genres.head(10))

genre_name
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515
Name: track_name, dtype: int64

**Вывод:** В Москве, кроме абсолютно популярного жанра поп, есть направление русской популярной музыки. Значит, что интерес к этому жанру шире. А рэп, вопреки предположению, занимает в обоих городах близкие позиции.

## Общие выводы по проекту

Мы ставили 3 рабочие гипотезы:
1. музыку в двух городах — Москве и Санкт-Петербурге — слушают в разном режиме;
2. списки десяти самых популярных жанров утром в понедельник и вечером в пятницу имеют характерные отличия;
3. население двух городов предпочитает разные музыкальные жанры.

Москва и Петербург сходятся во вкусах: везде преобладает популярная музыка. При этом зависимости предпочтений от дня недели в каждом отдельном городе нет — люди постоянно слушают то, что им нравится. Но между городами в разрезе дней неделей наблюдается зеркальность относительно среды: Москва больше слушает в понедельник и пятницу, а Петербург наоборот - больше в среду, но меньше в понедельник и пятницу.

В результате первая гипотеза подтверждена, вторая гипотеза не подтверждена и третья не подтверждена.

